# National, state, and county-level dealer trends

In [ ]:
import pandas as pd
import numpy as np
import altair as alt

In [ ]:
# These are the years of FFL lists that we have
years = ['2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
         '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', 
         '2023']

In [ ]:
national = pd.DataFrame()
state = pd.DataFrame()
county = pd.DataFrame()

In [ ]:
for year in years:
    df_national = pd.read_csv('../../data/processed/atf-ffl-list/'+year+'-ffl-list-license-types.csv')
    df_national['year'] = year
    national = pd.concat([national, df_national])
    df_state = pd.read_csv('../../data/processed/atf-ffl-list/'+year+'-ffl-list-states.csv', dtype={"statefp":'str'})
    df_state['year'] = year
    state = pd.concat([state, df_state])
    df_county = pd.read_csv('../../data/processed/atf-ffl-list/'+year+'-ffl-list-counties.csv', dtype={"fips":'str'})
    df_county['year'] = year
    county = pd.concat([county, df_county])

In [ ]:
national.to_csv('../../data/processed/atf-ffl-list/ffls_total_type.csv', index=False)
state.to_csv('../../data/processed/atf-ffl-list/ffls_state.csv', index=False)
county.to_csv('../../data/processed/atf-ffl-list/ffls_county.csv', index=False)

### Latest FFLs

In [ ]:
latest = pd.read_csv('../../data/processed/atf-ffl-list/1023-ffl-list-license-types.csv')

In [ ]:
latest

In [ ]:
latest["count"].sum()

### National FFLs

In [ ]:
national_totals = national.groupby(['year'])['count'].sum().reset_index()

In [ ]:
national_totals.to_csv('../../data/processed/atf-ffl-list/ffls_total.csv', index=False)

In [ ]:
national.groupby(['year','lic_type'])['count'].sum().reset_index().pivot(index='lic_type', columns='year', values='count').reset_index()

In [ ]:
national.groupby(['year'])['count'].sum().reset_index()

In [ ]:
alt.Chart(national).mark_area().encode(
    x="year:T",
    y="count:Q",
    color="lic_type:N"
)

In [ ]:
alt.Chart(national).mark_area().encode(
    x="year:T",
    y="count:Q",
    color="lic_type:N"
).properties(
    width=200,
    height=140
).facet('lic_type', columns=4)

### State FFLs

In [ ]:
state_group = state.groupby(['year','state'])['count'].sum().reset_index()

In [ ]:
state_group[state_group["state"] == "CA"]

In [ ]:
state_group_pivot = state_group.pivot(index='state', columns='year', values='count').reset_index()

In [ ]:
state_group_pivot["difference"] = state_group_pivot["2023"] - state_group_pivot["2003"]

In [ ]:
state_group_pivot["pct_difference"] = (state_group_pivot["2023"] - state_group_pivot["2003"]) / state_group_pivot["2003"]

In [ ]:
state_group_pivot.to_csv('../../data/processed/ffl-count-changes-by-state.csv', index=False)

### States that have increased the most

In [ ]:
state_group_pivot.sort_values("pct_difference", ascending=False)[["state","difference","pct_difference"]].head(10)

### States that have decreased the most

In [ ]:
state_group_pivot.sort_values("pct_difference", ascending=False)[["state","difference","pct_difference"]].tail(10)

### California FFLs

In [ ]:
ca = state.loc[state["state"] == "CA"]

In [ ]:
ca

In [ ]:
alt.Chart(ca).mark_area().encode(
    x="year",
    y="count",
)

### Southern California FFLs

In [ ]:
ca_counties = county.loc[(county["countyname"].isin(["Los Angeles County","Orange County","Riverside County", "Kern County", "Ventura County"]))& (county["state"] == "CA")]

In [ ]:
alt.Chart(ca_counties).mark_area().encode(
    x="year",
    y="count",
).properties(
    width=200,
    height=140
).facet('countyname', columns=4)